In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency
import imblearn
import sklearn
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [58]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.svm import SVC
from xgboost import XGBClassifier

In [59]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score , recall_score

from fairlearn.metrics import MetricFrame, false_positive_rate, false_negative_rate,count, selection_rate
from fairlearn.datasets import fetch_adult
from fairlearn.postprocessing import ThresholdOptimizer, plot_threshold_optimizer
from fairlearn.metrics import demographic_parity_ratio, equalized_odds_ratio
from fairlearn.reductions import DemographicParity
import pandas as pd
import json

In [2]:

train_data = pd.read_csv('data/train.csv')

In [3]:
# Remove quotes and replace spaces with underscores
train_data.columns = [col.strip().replace("'", '').replace(' ', '_') for col in train_data.columns]



In [4]:
train_data = train_data.rename(columns={
    'Genes_in_mothers_side': 'Genes_Mothers_Side',
    'Inherited_from_father': 'Inherited_Father',
    'Blood_cell_count_(mcL)': 'Blood_Cell_mcL',
    'Respiratory_Rate_(breaths/min)': 'Respiratory_Rate_breaths_min',
    'Heart_Rate_(rates/min': 'Heart_Rates_Min',
    'Follow-up': 'Follow_up',
    'Autopsy_shows_birth_defect_(if_applicable)': 'Autopsy_Birth_Defect',
    'Folic_acid_details_(peri-conceptional)': 'Folic_Acid',
    'H/O_serious_maternal_illness': 'Maternal_Illness',
    'H/O_radiation_exposure_(x-ray)': 'Radiation_Exposure',
    'H/O_substance_abuse': 'Substance_Abuse',
    'Assisted_conception_IVF/ART': 'Assisted_Conception',
    'History_of_anomalies_in_previous_pregnancies': 'History_Previous_Pregnancies',
    'No._of_previous_abortion': 'Previous_Abortion',
    'Birth_defects': 'Birth_Defects',
    'White_Blood_cell_count_(thousand_per_microliter)': 'White_Blood_Cell',
})

In [5]:
# Drop rows where 'Genetic_Disorder' is NaN
train_data.dropna(subset=['Genetic_Disorder'], inplace=True)

# Reset the index to update row numbers and avoid keeping the old index
train_data.reset_index(drop=True, inplace=True)

### Perform Chi-Square Test for Each Feature
    - to evaluate feature improtance

In [6]:
import pandas as pd
from scipy.stats import chi2_contingency

# Assuming train_data is already loaded
# Define X (all categorical features except the target) and y (target variable)
X = train_data.drop(columns=['Genetic_Disorder', 'Disorder_Subclass'])  # Features
y = train_data['Genetic_Disorder']  # Target variable


In [7]:
# Initialize an empty list to store results
chi2_results = []

# Loop through each feature in X
for column in X.columns:
    # Create a contingency table for the feature and the target
    contingency_table = pd.crosstab(X[column], y)
    
    # Perform the Chi-Square test
    chi2_stat, p, dof, expected = chi2_contingency(contingency_table)
    
    # Append results (feature name, Chi-Square statistic, p-value)
    chi2_results.append({'Feature': column, 'Chi2 Stat': chi2_stat, 'p-value': p})

# Convert results to a DataFrame for better visualization
chi2_df = pd.DataFrame(chi2_results)

# Sort features by Chi-Square statistic (descending order)
chi2_df = chi2_df.sort_values(by='Chi2 Stat', ascending=False)

print(chi2_df)


                         Feature     Chi2 Stat        p-value
0                     Patient_Id  39874.000000   4.962328e-01
6                 Blood_Cell_mcL  39874.000000   4.962328e-01
36              White_Blood_Cell  30969.304917   9.410793e-01
9                   Fathers_name  30507.493656   2.989873e-01
8                    Family_Name  12018.213711   3.519372e-01
7             Patient_First_Name   4988.118109   9.422057e-02
42                     Symptom_5   1001.441227  3.465776e-218
41                     Symptom_4    653.909808  1.012253e-142
40                     Symptom_3    587.264659  2.999850e-128
39                     Symptom_2    386.997596   9.215941e-85
38                     Symptom_1    210.217763   2.247982e-46
2             Genes_Mothers_Side    174.722576   1.146766e-38
3               Inherited_Father    172.798123   3.001680e-38
4                  Maternal_gene    114.615651   1.292789e-25
5                  Paternal_gene    112.651041   3.452529e-25
11      

In [8]:
train_data = train_data.drop(columns=['Patient_Id','Patient_First_Name', 'Family_Name',
       'Fathers_name','Institute_Name', 
       'Location_of_Institute','Test_1', 'Test_2', 'Test_3', 'Test_4','Test_5','Parental_consent'])
#, 
  #   

In [31]:
train_data.isnull().sum()

Patient_Age                     1225
Genes_Mothers_Side                 0
Inherited_Father                 261
Maternal_gene                   2424
Paternal_gene                      0
Blood_Cell_mcL                     0
Mothers_age                     5204
Fathers_age                     5165
Status                             0
Respiratory_Rate_breaths_min    1826
Heart_Rates_Min                 1805
Follow_up                       1862
Gender                          1844
Birth_asphyxia                  1848
Autopsy_Birth_Defect            3935
Place_of_birth                  1820
Folic_Acid                      1814
Maternal_Illness                1824
Radiation_Exposure              1830
Substance_Abuse                 1900
Assisted_Conception             1847
History_Previous_Pregnancies    1893
Previous_Abortion               1828
Birth_Defects                   1839
White_Blood_Cell                1849
Blood_test_result               1833
Symptom_1                       1844
S

In [32]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Define numeric and categorical features explicitly
numeric_features = [
    "Patient_Age", "Blood_Cell_mcL", "Mothers_age", "Fathers_age", 
    "Previous_Abortion", "White_Blood_Cell", "Symptom_1", 
    "Symptom_2", "Symptom_3", "Symptom_4", "Symptom_5"
]
categorical_features = [
    "Genes_Mothers_Side", "Inherited_Father", "Maternal_gene", 
    "Paternal_gene", "Status", "Respiratory_Rate_breaths_min", 
    "Heart_Rates_Min", "Follow_up", "Gender", "Birth_asphyxia", 
    "Autopsy_Birth_Defect", "Place_of_birth", "Folic_Acid", 
    "Maternal_Illness", "Radiation_Exposure", "Substance_Abuse", 
    "Assisted_Conception", "History_Previous_Pregnancies", 
    "Birth_Defects", "Blood_test_result"
]

# Preprocessor for X
numeric_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocessor_X = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# Preprocessor for y (Label Encoding)
label_encoder_y = LabelEncoder()




In [33]:
# Prepare features (X)
X = train_data.drop(columns=["Genetic_Disorder", "Disorder_Subclass"])
X_preprocessed = preprocessor_X.fit_transform(X)

# Prepare target (y)
y = train_data["Genetic_Disorder"]
y_preprocessed = label_encoder_y.fit_transform(y)


In [37]:
# Assuming X_preprocessed is a numpy array and column names are stored
import numpy as np

# Check for nulls in each column
nulls_per_column = np.isnan(X_preprocessed).sum(axis=0)
print("Null values per column:\n", nulls_per_column)


Null values per column:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [38]:
# Check for null values in the target variable
print(y.isnull().sum())

# Inspect unique values in the target variable
print(y.unique())


0
['Mitochondrial genetic inheritance disorders'
 'Multifactorial genetic inheritance disorders'
 'Single-gene inheritance diseases']


In [39]:
label_encoder_y.fit(["Mitochondrial genetic inheritance disorders", "Multifactorial genetic inheritance disorders", "Single-gene inheritance diseases'"])  # Original classes used in encoding

# Reverse transform to get original labels
y_decoded = label_encoder_y.inverse_transform(y_preprocessed)
print(y_decoded)

['Mitochondrial genetic inheritance disorders'
 'Multifactorial genetic inheritance disorders'
 'Mitochondrial genetic inheritance disorders' ...
 'Multifactorial genetic inheritance disorders'
 'Mitochondrial genetic inheritance disorders'
 'Multifactorial genetic inheritance disorders']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y_preprocessed, test_size=0.2, random_state=42)

In [ ]:
# Example: Training a Logistic Regression model without class balancing or feature selection 
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Model evaluation
score = model.score(X_test, y_test)
print(f"Model accuracy: {score:.2f}")

# Optional: Decode predictions for interpretability
y_pred = model.predict(X_test)
y_pred_decoded = label_encoder_y.inverse_transform(y_pred)

# Display decoded predictions
print("Decoded predictions:", y_pred_decoded[:10])

# Feature selection methods 
 - Chi-Square test already implemented

In [47]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {"C": [0.01, 0.1, 1, 10, 100]}

# Grid search for optimal C
grid_search = GridSearchCV(
    LogisticRegression(penalty="l1", solver="liblinear", max_iter=1000),
    param_grid,
    cv=5,
    scoring="accuracy"
)
grid_search.fit(X_train, y_train)

# Get best parameters and evaluate
best_model = grid_search.best_estimator_
score = best_model.score(X_test, y_test)
print(f"Best regularization strength (C): {grid_search.best_params_['C']}")
print(f"Model accuracy after tuning: {score:.2f}")


Best regularization strength (C): 0.01
Model accuracy after tuning: 0.56


In [48]:
from sklearn.linear_model import Lasso 
# Fit Lasso regression model
lasso = Lasso(alpha=0.01)  # Alpha controls the regularization strength
lasso.fit(X_train, y_train)

feature_names = preprocessor_X.get_feature_names_out()
# Extract important features
coefficients = lasso.coef_
important_features = [
    (feature, coef) for feature, coef in zip(feature_names, coefficients) if coef != 0
]

# Sort features by importance
important_features = sorted(important_features, key=lambda x: abs(x[1]), reverse=True)

# Display the top features
print("Top important features (Lasso):")
for feature, coef in important_features[:10]:
    print(f"{feature}: {coef:.4f}")

Top important features (Lasso):
num__Symptom_2: -0.0069
num__Symptom_4: -0.0019


In [49]:
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Feature Importance
feature_importances = rf.feature_importances_
important_features = sorted(
    zip(feature_names, feature_importances),
    key=lambda x: x[1],
    reverse=True
)

# Display top features
print("Top important features (Random Forest):")
for feature, importance in important_features[:10]:
    print(f"{feature}: {importance:.4f}")


Top important features (Random Forest):
num__Symptom_5: 0.0491
num__Blood_Cell_mcL: 0.0458
num__White_Blood_Cell: 0.0434
num__Symptom_3: 0.0405
num__Fathers_age: 0.0396
num__Mothers_age: 0.0382
num__Patient_Age: 0.0380
num__Symptom_4: 0.0369
num__Previous_Abortion: 0.0283
num__Symptom_2: 0.0262


In [ ]:
from sklearn.feature_selection import mutual_info_classif

# Calculate mutual information
mutual_info = mutual_info_classif(X_preprocessed, y_preprocessed, discrete_features=True)

# Combine features with scores
important_features = sorted(
    zip(feature_names, mutual_info),
    key=lambda x: x[1],
    reverse=True
)

# Display top features
print("Top important features (Mutual Information):")
for feature, score in important_features[:10]:
    print(f"{feature}: {score:.4f}")


# Implimenting SMOTE for class balancing

In [ ]:
import numpy as np

# Count target class distribution
unique_classes, counts = np.unique(y_preprocessed, return_counts=True)
for cls, count in zip(unique_classes, counts):
    print(f"Class {cls}: {count} instances")


Class 0: 10202 instances
Class 1: 2071 instances
Class 2: 7664 instances


In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_preprocessed, y_preprocessed)

# Train-test split after resampling
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


In [ ]:
import numpy as np

# Count target class distribution
unique_classes, counts = np.unique(y_resampled, return_counts=True)
for cls, count in zip(unique_classes, counts):
    print(f"Class {cls}: {count} instances")


Class 0: 10202 instances
Class 1: 10202 instances
Class 2: 10202 instances


In [51]:
selected_features = [
    "num__Blood_Cell_mcL", "num__White_Blood_Cell", "num__Symptom_5",
    "num__Symptom_4", "num__Symptom_3", "num__Symptom_2", "num__Symptom_1",
    "num__Fathers_age", "num__Mothers_age", "num__Patient_Age",
    "cat__Blood_test_result_slightly abnormal", "cat__Birth_Defects_Multiple",
    "cat__History_Previous_Pregnancies_No"
]

X_resampled_selected = X_resampled[:, [feature_names.tolist().index(f) for f in selected_features]]



In [52]:
# Train-test split after feature selection
X_train, X_test, y_train, y_test = train_test_split(X_resampled_selected, y_resampled, test_size=0.2, random_state=42)

In [53]:
print("Number of features after selection:", X_resampled_selected.shape[1])
print("Number of selected features:", len(selected_features))


Number of features after selection: 13
Number of selected features: 13


In [54]:
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)


Training set shape: (24484, 13)
Testing set shape: (6122, 13)


## Implementing models

In [55]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Train a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced")
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))


Random Forest Accuracy: 0.6891538712838942
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.68      0.65      2052
           1       0.80      0.90      0.84      2024
           2       0.63      0.50      0.55      2046

    accuracy                           0.69      6122
   macro avg       0.68      0.69      0.68      6122
weighted avg       0.68      0.69      0.68      6122



In [56]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

# Train One-vs-All Logistic Regression
ovr_model = OneVsRestClassifier(LogisticRegression(max_iter=1000))
ovr_model.fit(X_train, y_train)

# Evaluate
y_pred_ovr = ovr_model.predict(X_test)
print("One-vs-All Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_ovr))


One-vs-All Logistic Regression Accuracy: 0.5360993139496897


In [57]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

# Define base models
base_models = [
    ("random_forest", RandomForestClassifier(n_estimators=100, random_state=42)),
    ("gradient_boosting", GradientBoostingClassifier(random_state=42))
]

# Define meta-model
meta_model = LogisticRegression(max_iter=1000)

# Define stacking classifier
stacked_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)
stacked_model.fit(X_train, y_train)

# Evaluate
y_pred_stacked = stacked_model.predict(X_test)
print("Stacked Model Accuracy:", accuracy_score(y_test, y_pred_stacked))
print("Classification Report:\n", classification_report(y_test, y_pred_stacked))


Stacked Model Accuracy: 0.7018948056190787
Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.72      0.67      2052
           1       0.83      0.87      0.85      2024
           2       0.63      0.52      0.57      2046

    accuracy                           0.70      6122
   macro avg       0.70      0.70      0.70      6122
weighted avg       0.70      0.70      0.70      6122



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Define diverse base models
base_estimators = [
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(probability=True, random_state=42))
]

# Stacked model
stacked_model_diverse = StackingClassifier(
    estimators=base_estimators,
    final_estimator=LogisticRegression(),
    cv=5
)

# Train and evaluate
stacked_model_diverse.fit(X_train, y_train)
y_pred_stacked_diverse = stacked_model_diverse.predict(X_test)
print("Stacked Model Accuracy (XGBoost + RF + SVC):", accuracy_score(y_test, y_pred_stacked_diverse))


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(stacked_model, X_balanced, y_balanced, cv=5, scoring='accuracy')
print("Cross-validated accuracy (GB + XGBoost):", scores.mean())


### Implimenting  base classifiers for multi classifier apperach - All models separately 

In [60]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
# Define base models
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
gb_model = GradientBoostingClassifier(random_state=42)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
svc_model = SVC(probability=True, random_state=42)  # SVC with probability enabled

# Train and evaluate each model
models = [rf_model, gb_model, xgb_model, svc_model]
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"Accuracy of {model.__class__.__name__}: {accuracy_score(y_test, y_pred):.2f}")
    print(classification_report(y_test, y_pred))


Accuracy of RandomForestClassifier: 0.69
              precision    recall  f1-score   support

           0       0.63      0.68      0.65      2052
           1       0.79      0.89      0.84      2024
           2       0.62      0.50      0.55      2046

    accuracy                           0.69      6122
   macro avg       0.68      0.69      0.68      6122
weighted avg       0.68      0.69      0.68      6122

Accuracy of GradientBoostingClassifier: 0.64
              precision    recall  f1-score   support

           0       0.58      0.78      0.67      2052
           1       0.72      0.83      0.77      2024
           2       0.60      0.30      0.40      2046

    accuracy                           0.64      6122
   macro avg       0.63      0.64      0.61      6122
weighted avg       0.63      0.64      0.61      6122

Accuracy of XGBClassifier: 0.66
              precision    recall  f1-score   support

           0       0.60      0.71      0.65      2052
           

In [61]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Define base models
base_estimators = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingClassifier(random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
    ('svc', SVC(probability=True, random_state=42))  # Enable probability for SVC
]

# Stacked model with Logistic Regression as the meta-classifier
stacked_model = StackingClassifier(
    estimators=base_estimators,
    final_estimator=LogisticRegression(max_iter=1000),
    cv=5  # 5-fold cross-validation for meta-model training
)

# Train the stacked model
stacked_model.fit(X_train, y_train)

# Make predictions
y_pred_stacked = stacked_model.predict(X_test)

# Evaluate the stacked model
accuracy = accuracy_score(y_test, y_pred_stacked)
print(f"Stacked Model Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred_stacked))


Stacked Model Accuracy: 0.70
Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.73      0.68      2052
           1       0.84      0.86      0.85      2024
           2       0.63      0.51      0.56      2046

    accuracy                           0.70      6122
   macro avg       0.70      0.70      0.70      6122
weighted avg       0.70      0.70      0.70      6122



Targeted oversampling for class 2 

In [65]:
print("Unique classes in y_preprocessed:", set(y_preprocessed))


Unique classes in y_preprocessed: {0, 1, 2}


In [73]:
smote = SMOTE(sampling_strategy={0: 3000, 1: 3000, 2: 4000}, random_state=42)


In [74]:
print("X_preprocessed shape:", X_preprocessed.shape)
print("y_preprocessed shape:", y_preprocessed.shape)


X_preprocessed shape: (19937, 61)
y_preprocessed shape: (19937,)


In [75]:
# Correct SMOTE configuration
#smote = SMOTE(sampling_strategy='auto', random_state=42)

# Apply SMOTE
X_resampled, y_resampled = smote.fit_resample(X_preprocessed, y_preprocessed)

# Verify resampled data
print("Resampled X shape:", X_resampled.shape)
print("Resampled y shape:", y_resampled.shape)
print("Class distribution after resampling:", pd.Series(y_resampled).value_counts())


ValueError: With over-sampling methods, the number of samples in a class should be greater or equal to the original number of samples. Originally, there is 10202 samples and 3000 samples are asked.

In [69]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy={2: 4000}, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_preprocessed, y_preprocessed)


ValueError: With over-sampling methods, the number of samples in a class should be greater or equal to the original number of samples. Originally, there is 7664 samples and 4000 samples are asked.

In [ ]:
selected_features = [
    "num__Blood_Cell_mcL", "num__White_Blood_Cell", "num__Symptom_5",
    "num__Symptom_4", "num__Symptom_3", "num__Symptom_2", "num__Symptom_1",
    "num__Fathers_age", "num__Mothers_age", "num__Patient_Age",
    "cat__Blood_test_result_slightly abnormal", "cat__Birth_Defects_Multiple",
    "cat__History_Previous_Pregnancies_No"
]

X_resampled_selected = X_resampled[:, [feature_names.tolist().index(f) for f in selected_features]]


# Train-test split after feature selection
X_train, X_test, y_train, y_test = train_test_split(X_resampled_selected, y_resampled, test_size=0.2, random_state=42)

In [ ]:
import numpy as np

# Count target class distribution
unique_classes, counts = np.unique(y_resampled, return_counts=True)
for cls, count in zip(unique_classes, counts):
    print(f"Class {cls}: {count} instances")


In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

def evaluate_stacked_model(X_train, y_train, X_test, y_test):
    # Define base models
    base_estimators = [
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('gb', GradientBoostingClassifier(random_state=42)),
        ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
        ('svc', SVC(probability=True, random_state=42))  # Enable probability for SVC
    ]

    # Define the stacked model
    stacked_model = StackingClassifier(
        estimators=base_estimators,
        final_estimator=LogisticRegression(max_iter=1000),
        cv=5  # 5-fold cross-validation for meta-model training
    )

    # Train and evaluate the stacked model
    stacked_model.fit(X_train, y_train)
    y_pred_stacked = stacked_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_stacked)
    print(f"Stacked Model Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", classification_report(y_test, y_pred_stacked))
    
    return stacked_model, accuracy


In [ ]:
# Evaluate the stacked model
stacked_model, stacked_accuracy = evaluate_stacked_model(X_train, y_train, X_test, y_test)


In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

def evaluate_individual_models(X_train, y_train, X_test, y_test):
    # Define base models
    models = [
        RandomForestClassifier(n_estimators=100, random_state=42),
        GradientBoostingClassifier(random_state=42),
        XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
        SVC(probability=True, random_state=42)  # Enable probability for SVC
    ]

    # Train and evaluate each model
    results = {}
    for model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy of {model.__class__.__name__}: {accuracy:.2f}")
        print(classification_report(y_test, y_pred))
        results[model.__class__.__name__] = accuracy
    
    return results


In [ ]:
# Evaluate individual models
individual_results = evaluate_individual_models(X_train, y_train, X_test, y_test)


In [83]:
from skmultilearn.problem_transform import ClassifierChain

In [84]:
# Chain Classifier using Random Forest
from skmultilearn.problem_transform import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

chain_classifier = ClassifierChain(RandomForestClassifier(n_estimators=100, random_state=42))
chain_classifier.fit(X_train, y_train)

# Convert predictions to dense format
y_pred_chain = chain_classifier.predict(X_test).toarray()

# If y_test is sparse, convert it to dense format too
if hasattr(y_test, 'toarray'):
    y_test_dense = y_test.toarray()
else:
    y_test_dense = y_test

# Evaluate the Chain Classifier
print("Chain Classifier Accuracy:", accuracy_score(y_test_dense, y_pred_chain))
print(classification_report(y_test_dense, y_pred_chain))


Chain Classifier Accuracy: 0.6868670369160406
              precision    recall  f1-score   support

           0       0.63      0.68      0.65      2052
           1       0.79      0.89      0.84      2024
           2       0.62      0.50      0.55      2046

    accuracy                           0.69      6122
   macro avg       0.68      0.69      0.68      6122
weighted avg       0.68      0.69      0.68      6122



In [ ]:
param_grid = {
    'base_estimator__n_estimators': [10, 50, 100, 200],
    'base_estimator__max_depth': [None, 5, 10, 15],
    'base_estimator__min_samples_split': [2, 5, 10],
    'base_estimator__min_samples_leaf': [1, 5, 10]
}

grid_search = GridSearchCV(ClassifierChain(RandomForestClassifier(random_state=42)), param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

best_chain_classifier = grid_search.best_estimator_
y_pred_chain = best_chain_classifier.predict(X_test)

print("Chain Classifier Accuracy:", accuracy_score(y_test, y_pred_chain))
print("Classification Report:")
print(classification_report(y_test, y_pred_chain))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_chain))